# Some Restaurant Data Manipulations

In [ ]:
import pandas as pd
import matplotlib

In [ ]:
file_inspec = pd.read_csv("data/food_establishment_inspections.csv", engine='python')

In [ ]:
file_311 = pd.read_csv("data/311_service_requests.csv", low_memory= False)

In [ ]:
# Get # of different types of Food complaints
complaint_types = file_311["complaint_type"].unique()
print(complaint_types)
calorie_counts = file_311.loc[file_311['complaint_type'] == 'Calorie Labeling']
print(calorie_counts.shape)
food_est = file_311.loc[file_311['complaint_type'] == 'Food Establishment']
print(food_est.shape)
food_poi = file_311.loc[file_311['complaint_type'] == 'Food Poisoning']
print(food_poi.shape)

In [116]:
# Count 311 complaints and food inspections for each restaurant
def inspecKey(row):
    return str(row['latitude']) + "," + str(row['longitude'])
def estKey(row):
    return str(row['latitude']) + "," + str(row['longitude'])

restaurants = {} # Key = lat/long b/c unique for each restaurant

# Count how many inspections each restaurant has had
for index, row in file_inspec.iterrows():
    key = inspecKey(row)
    if key in restaurants:
        restaurants[key]['inspec'] += 1
    else:
        restaurants[key] = { 'inspec' : 0, '311' : 0 }

# Count how many 311's each restaurant has had
overlapCount = 0
for index, row in food_est.iterrows():
    key = estKey(row)
    if key in restaurants:
        if restaurants[key]['311'] == 0:
            overlapCount += 1
        restaurants[key]['311'] += 1
    else:
        restaurants[key] = { 'inspec' : 0, '311' : 0 }

print(overlapCount)

12780


# NTA Demographics

In [ ]:
# Read demographics_city file (given by Citadel) + 4 ACS supplemental files
file_nta_demo = pd.read_csv("data/demographics_city.csv")
supp_nta_demo = pd.read_excel("data/demographic_supp/nta_demo.xlsx")
supp_nta_econ = pd.read_excel("data/demographic_supp/nta_econ.xlsx")
supp_nta_hous = pd.read_excel("data/demographic_supp/nta_hous.xlsx")
supp_nta_soc = pd.read_excel("data/demographic_supp/nta_soc.xlsx")

In [ ]:
# There is a slight under-reporting of NTAs in the Citadel demographic file
# So get list of NTAs present in ASC survey but not in demographics_city
for index, row in supp_nta_demo.iterrows():
    if row['GeoID'] not in file_nta_demo['nta_code'].tolist():
        print(row['GeoID'])

In [ ]:
# Merge together all dataframes with NTA data
nta_merged = pd.merge(file_nta_demo, pd.merge(supp_nta_soc, pd.merge(pd.merge(supp_nta_econ, supp_nta_demo, on = 'GeoID'), supp_nta_hous, on = 'GeoID')), left_on = 'nta_code', right_on = 'GeoID', how='left')
print(nta_merged.head())

In [ ]:
# Determine most commonly used non-English languages by New Yorkers
langs = ['Sp', 'Fr', 'FrC', 'It', 'Prt', 'Grm', 'Yid', 'OWG', 'Scd', 'Grk', 'Rus', 'Pol', 'SCr', 'OSv', 'Arm', 'Prs', 'Guj', 'Hdi', 'Urd', 'OIn', 'OIE', 'Chi', 'Jap', 'Kor', 'MKm', 'Hmg', 'Tha', 'Lao', 'Vie', 'OAn', 'Tag', 'OPI', 'Nav', 'ONA', 'Hng', 'Arb', 'Heb', 'Afr',]
soc_langs = []
for l in langs:
    soc_langs.append({ 'lang' : l, 'val' : supp_nta_soc['Lg'+l+'LEP2E'].sum()})

print(sorted(soc_langs, key= lambda x : x['val'], reverse = True))

In [ ]:
# Store interesting NTA demographic data in dictionary
nta_demographics = {}
nta_demographics_dataframe_prototype = []
for index, row in nta_merged.iterrows():
    nta_demographics[row['nta_code']] = {
        'pop' : row['population'],
        'households' : row['households'],
        'ages' : [ 
            row['under_5_years'], row['5-9_years'], row['10-14_years'], row['15-19_years'], row['20-24_years'], 
            row['25-29_years'], row['30-34_years'], row['35-39_years'], row['40-44_years'], row['45-49_years'], 
            row['50-54_years'], row['55-59_years'], row['60-64_years'], row['over_65_years'],
        ],
        'median_hh_income' : row['median_income'],
        'hh_incomes' : [ # Number of house holds in given income bracket
            row['less_than_10,000'], row['10000_to_14999'], row['15000_to_24999'], row['25000_to_34999'], 
            row['35000_to_49999'], row['50000_to_74999'], row['75000_to_99999'], row['100000_to_149999'], 
            row['150000_to_199999'], row['200000_or_more'],
        ],
        'below_poverty' : row['PBwPvE'], # Number of people living below poverty line
        'education_level' : [ # 25+ years, highest level of education attained
            int(row['EA_LTHSGrE']), # No high school
            int(row['EA_HScGrdE']), # High school grad
            int(row['EA_AscDE'])+int(row['EA_BchDE']) + int(row['EA_GrdPfDE']), # College/graduate degree
        ],
        'commute' : { # How ppl commute to work
            'walk' : row['CW_WlkdE'],
            'public' : row['CW_PbTrnsE'],
            'car' : int(row['CW_CrpldE'])+int(row['CW_DrvAlnE']),
        },
        'health_insurance_coverage' : row['HInsE'], # Number of ppl with public or private health insurance
        'disability' : row['CvNIDE'], # Number of ppl living with disability
        'not_us_citizen' : row['FbNotCznE'], # Number of non-US citizens
        'languages' : { # Number of ppl who speak each language and do not speak English "very well" - Note: These are the top 6 such languages in NYC
            'spanish' : row['LgSpLEP2E'],
            'chinese' : row['LgChiLEP2E'],
            'russian' : row['LgRusLEP2E'],
            'indic' : row['LgOInLEP2E'],
            'french_creole' : row['LgFrCLEP2E'],
            'korean' : row['LgKorLEP2E'],
        },
        'home_value' : [
            row['VlU50E'], # <50k
            row['Vl50t99E'], # 50-99k
            row['Vl100t149E'], # 100-149k
            row['Vl150t199E'], # 150-199k
            row['Vl200t299E'], # 200-299k
            row['Vl300t499E'], # 300-499k
            row['Vl500t999E'], # 500-999k
            row['Vl1milplE'], # 1M + 
        ],
        'rent' : [
            row['GRU500E'], # <500
            row['GR500t999E'], # 500-999
            row['GR1kt14kE'], # 1-1.5k
            row['GR15kt19kE'], # 1.5-2k
            row['GR20kt24kE'], # 2-2.5k
            row['GR25kt29kE'], # 2.5-3k
            row['GR3kplE'], # 3k+
        ],
        'average_hh_size' : row['AvgHHSzE'],
    }
    nta_demographics_dataframe_prototype.append([
        row['nta_code'],
        row['population'], row['households'],
        row['under_5_years'], row['5-9_years'], row['10-14_years'], row['15-19_years'], row['20-24_years'], row['25-29_years'], row['30-34_years'], row['35-39_years'], row['40-44_years'], row['45-49_years'], row['50-54_years'], row['55-59_years'], row['60-64_years'], row['over_65_years'],
        row['median_income'],
        row['less_than_10,000'], row['10000_to_14999'], row['15000_to_24999'], row['25000_to_34999'], row['35000_to_49999'], row['50000_to_74999'], row['75000_to_99999'], row['100000_to_149999'], row['150000_to_199999'], row['200000_or_more'],
        row['PBwPvE'],
        row['EA_LTHSGrE'], row['EA_HScGrdE'], int(row['EA_AscDE'])+int(row['EA_BchDE']) + int(row['EA_GrdPfDE']),
        row['CW_WlkdE'], row['CW_PbTrnsE'], int(row['CW_CrpldE'])+int(row['CW_DrvAlnE']),
        row['HInsE'],
        row['CvNIDE'],
        row['FbNotCznE'],
        row['LgSpLEP2E'], row['LgChiLEP2E'], row['LgRusLEP2E'], row['LgOInLEP2E'], row['LgFrCLEP2E'], row['LgKorLEP2E'],
        row['VlU50E'], row['Vl50t99E'], row['Vl100t149E'], row['Vl150t199E'], row['Vl200t299E'], row['Vl300t499E'], row['Vl500t999E'], row['Vl1milplE'],
        row['GRU500E'], row['GR500t999E'], row['GR1kt14kE'], row['GR15kt19kE'], row['GR20kt24kE'], row['GR25kt29kE'], row['GR3kplE'], row['AvgHHSzE'],
    ])

print(nta_demographics)

In [119]:
# NTA dictionary -> Pandas Dataframe
nta_dataframe = pd.DataFrame(nta_demographics_dataframe_prototype,
                                      columns = ['nta_code', 'pop', 'households', 'age_less_5', 'age_5_to_9', 'age_10_to_14', 'age_15_to_19', 'age_20_to_24', 'age_25_to_29', 'age_30_to_34', 'age_35_to_39', 'age_40_to_44', 'age_45_to_49', 'age_50_to_54', 'age_55_to_59', 'age_60_to_64', 'age_65_plus', 'median_hh_income', 'hh_income_less_10', 'hh_income_10_to_15', 'hh_income_15_to_25', 'hh_income_25_to_35', 'hh_income_35_to_50', 'hh_income_50_to_75', 'hh_income_75_to_100', 'hh_income_100_to_150', 'hh_income_150_to_200', 'hh_income_200_plus', 'below_poverty', 'edu_none', 'edu_high_school', 'edu_college', 'commute_walk', 'commute_public', 'commute_car', 'health_insurance_coverage', 'disability', 'not_us_citizen', 'lang_spanish', 'lang_chinese', 'lang_russian', 'lang_indic', 'lang_french_creole', 'lang_korean', 'homeval_less_50', 'homeval_50_to_100', 'homeval_100_to_150', 'homeval_150_to_200', 'homeval_200_to_300', 'homeval_300_to_500', 'homeval_500_to_1m', 'homeval_1m_plus', 'rent_less_500', 'rent_500_to_1000', 'rent_1000_to_1500', 'rent_1500_to_2000', 'rent_2000_to_2500', 'rent_2500_to_3000', 'rent_3000_plus', 'average_hh_size', ])
print(nta_dataframe.head())

nta_dataframe.to_csv('nta_data.csv')

  nta_code    pop  households  age_less_5  age_5_to_9  age_10_to_14  \
0     BX31  28903        9587        1679        1706          1763   
1     SI01  27770        9885        1397        1698          1817   
2     SI48  25238        8960        1507        1540          1596   
3     QN70  78793       34310        3480        3037          3060   
4     QN48  19996        7122         917         966          1063   

   age_15_to_19  age_20_to_24  age_25_to_29  age_30_to_34       ...         \
0          2039          1964          1703          1798       ...          
1          1880          1720          1594          1487       ...          
2          1752          1614          1561          1726       ...          
3          3392          6630         11586          9252       ...          
4          1168          1214          1307          1296       ...          

   homeval_500_to_1m  homeval_1m_plus  rent_less_500  rent_500_to_1000  \
0               1644          

# County Demographics

In [ ]:
# Read Citadel's demographic file + 
file_county_demo = pd.read_csv("data/demographics_state.csv")
# supp_county_demo = pd.read_excel("data/demographic_supp/county_demo.xlsx")
# supp_county_econ = pd.read_excel("data/demographic_supp/county_econ.xlsx")
# supp_county_hous = pd.read_excel("data/demographic_supp/county_hous.xlsx")
# supp_county_soc = pd.read_excel("data/demographic_supp/county_soc.xlsx")

In [ ]:
# Citadel reports years 2011-16
# Filter file to only take data from the year 2016 (most recent data)
file_county_demo = file_county_demo.loc[file_county_demo["year"] == 2016]

In [ ]:
# Merge together all dataframes with county data
county_merged = file_county_demo
# pd.merge(file_county_demo, pd.merge(supp_nta_soc, pd.merge(pd.merge(supp_nta_econ, supp_nta_demo, on = 'GeoID'), supp_nta_hous, on = 'GeoID')), left_on = 'nta_code', right_on = 'GeoID', how='left')
# print(county_merged.head())

In [120]:
# Store interesting county demographic data in dictionary
county_demographics = {}
county_demographics_pandas_prototype = []
for index, row in county_merged.iterrows():
    county = row['geography']
    if county == "New York": 
        continue # Skip state info
    else: 
        county = county.replace(", New York", "") # Remove ", New York" from county name
    county_demographics[county] = {
        'pop': row['population'],
        'households' : row['total_households'],
        'hh_incomes' : {
            row['$9,999_or_less'], row['$10,000_to_$14,999'], row['$15,000_to_$24,999'], 
            row['$25,000_to_$34,999'], row['$35,000_to_$49,999'], row['$50,000_to_$74,999'],
            row['$75,000_to_$99,999'], row['$100,000_to_$150,000'], row['$150,000_to_$199,999'], 
            row['$200,000_or_more'],
        },
        'median_hh_income' : row['median_household_income'],
        'mean_soc_sec_income' : row['mean_soc_sec'], # Mean Social Security income
        'food_stamps' : row['food_stamp_benefits'], # Total SNAP benefits
        'health_insurance_coverage' : row['pop_w_health_insurance'],
    
    
    }
    county_demographics_pandas_prototype.append([
        county,
        row['population'],
        row['total_households'],
        row['$9,999_or_less'], row['$10,000_to_$14,999'], row['$15,000_to_$24,999'], 
        row['$25,000_to_$34,999'], row['$35,000_to_$49,999'], row['$50,000_to_$74,999'],
        row['$75,000_to_$99,999'], row['$100,000_to_$150,000'], row['$150,000_to_$199,999'], 
        row['$200,000_or_more'],
        row['median_household_income'],
        row['mean_soc_sec'],
        row['food_stamp_benefits'],
        row['pop_w_health_insurance'],
    ])

print(county_demographics)

{'Albany County': {'pop': 304048.0, 'households': 124108, 'hh_incomes': {19466, 6285, 22510, 8093, 16017, 11507, 14900, 7606, 10845, 6879}, 'median_hh_income': 60904, 'mean_soc_sec_income': 19131, 'food_stamps': 14305, 'health_insurance_coverage': 287725.0}, 'Allegany County': {'pop': 47207.0, 'households': 18032, 'hh_incomes': {320, 3521, 1282, 2308, 2277, 1604, 1193, 3196, 221, 2110}, 'median_hh_income': 44085, 'mean_soc_sec_income': 17561, 'food_stamps': 2872, 'health_insurance_coverage': 43052.0}, 'Bronx County': {'pop': 1414859.0, 'households': 490740, 'hh_incomes': {12865, 9410, 76719, 44111, 74323, 40947, 45398, 65338, 66942, 54687}, 'median_hh_income': 35302, 'mean_soc_sec_income': 14211, 'food_stamps': 179811, 'health_insurance_coverage': 1240816.0}, 'Broome County': {'pop': 194413.0, 'households': 78738, 'hh_incomes': {9066, 4972, 8560, 11189, 6134, 2742, 9432, 14361, 2203, 10079}, 'median_hh_income': 47744, 'mean_soc_sec_income': 18249, 'food_stamps': 12867, 'health_insuranc

In [121]:
# County dictionary -> Pandas Dataframe
county_dataframe = pd.DataFrame(county_demographics_pandas_prototype,
                                      columns = ['county_name', 'population', 'total_households', 'hh_income_less_10', 'hh_income_10_to_15', 'hh_income_15_to_25', 'hh_income_25_to_35', 'hh_income_35_to_50', 'hh_income_50_to_75', 'hh_income_75_to_100', 'hh_income_100_to_150', 'hh_income_150_to_200', 'hh_income_200_plus', 'median_household_income', 'mean_soc_sec', 'food_stamp_benefits', 'pop_w_health_insurance',])
print(county_dataframe.head())
county_dataframe.to_csv('county_data.csv')

          county_name  population  total_households  hh_income_less_10  \
0       Albany County    304048.0            124108               7606   
1     Allegany County     47207.0             18032               1282   
2        Bronx County   1414859.0            490740              76719   
3       Broome County    194413.0             78738               6134   
4  Cattaraugus County     77748.0             31502               2078   

   hh_income_10_to_15  hh_income_15_to_25  hh_income_25_to_35  \
0                6285               11507               10845   
1                1193                2308                2110   
2               45398               66942               54687   
3                4972               10079                8560   
4                2224                4272                4040   

   hh_income_35_to_50  hh_income_50_to_75  hh_income_75_to_100  \
0               14900               22510                16017   
1                3196           

# Brainstorming

Variable (file:column) POSITIVE/NEGATIVE Correlation - Rationale

- Demographics
    - Number of people (demographics_city:population) NEGATIVE - more people needing service means higher prioritization, 311 system more used to deploying in these areas
    - Population density (demographics_city:people_per_acre), NEGATIVE - more people means higher prioritization, denser areas in city are wealthier, more bang for buck in terms of resource allocation when servicing denser rather than sparse areas 
    - Median age (demographics_city:median_age)
    - Median income (demographics_city:median_income)
    - 5-yr age brackets (demographics_city:age_brackets)
    - Distribution of HH income (demographics_city:income_range), NEGATIVE - higher incomes get more gov service


Cleaning the supp files:
    - Note that there were 7 NTAs present in the ACS 2016 dataset for NYC that weren't present in the demographics_city.csv file. Looking at a map of NYC's NTAs, the blocks were: BX99 ("parks, cemeteries, etc." in the Bronx), BX98 (Riker's Island), MN99 ("parks, cemeteries, etc." in Manhattan), QN98 ("airports"), QN99 ("parks, cemeteries, etc." in Queens), and SI99 ("parks, cemeteries, etc." in Staten Island)